In [ ]:
import cv2
import sys
import numpy as np

def centreBoundingBox(vid_mid, bbox_mid):
    returnString = ""
    
    # Left / right correction
    if (bbox_mid[0] > vid_mid[0]) : # drone is leftwards of tree
        returnString = "Move right"
    elif (bbox_mid[0] < vid_mid[0]): # drone is rightwards of tree
        returnString = "Move left"  
    else:
        returnString = ""
        
    # Up / down correction
    if (bbox_mid[1] > vid_mid[1]): # drone is "above" tree
        returnString += " and down"
    elif (bbox_mid[1] < vid_mid[1]): # drone is "below" tree
        returnString += " and up"
        
    return returnString
 
if __name__ == '__main__' :
    
    # Set up tracker.
    tracker = cv2.TrackerCSRT_create()
    tracker_type = 'CSRT'

# Other tracking algorithm options... 
#     tracker = cv2.TrackerMIL_create()
#     tracker = cv2.TrackerMedianFlow_create()
 
    # Read video
    video = cv2.VideoCapture("Videos/IMG_9255.mp4")
 
    # Exit if video not opened.
    if not video.isOpened():
        print ("Could not open video")
        sys.exit()
 
    # Read first frame.
    ok, frame = video.read()
    if not ok:
        print ('Cannot read video file')
        
     
    # Define an initial bounding box (x,y,dx,dy)
    # Crucial that the bounding box is big enough
    
    #bbox = (796, 55, 40, 40) # for IMG_9238
    bbox = (467, 334, 50, 50) # for IMG_9255
 
    # Uncomment the line below to select a different bounding box
#     bbox = cv2.selectROI(frame, False)
#     print(bbox)
 
    # Initialize tracker with first frame and bounding box
    ok = tracker.init(frame, bbox)
    vid_height, vid_width, channels = frame.shape
    vid_mid_x = (int)(vid_width/2)
    vid_mid_y = (int)(vid_height/2)
    vid_mid = (vid_mid_x, vid_mid_y)
    print(vid_mid) 
    
    while True:
        # Read a new frame
        ok, frame = video.read()
        if not ok:
            break
         
        # Start timer
        timer = cv2.getTickCount()
 
        # Update tracker
        ok, bbox = tracker.update(frame)
 
        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
 
        # Draw bounding box
        if ok:
            # Tracking success
            
            # Find centre of new bounding box
            bbox_mid_x = bbox[0] + (int)(bbox[2]/2)
            bbox_mid_y = bbox[1] + (int)(bbox[3]/2)
            bbox_mid = (bbox_mid_x, bbox_mid_y) 
            
            # Move drone to get bounding box into the centre of the frame
            movement = centreBoundingBox(vid_mid, bbox_mid)
            
            # Display movement required
            cv2.putText(frame, movement, (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,50,250),2);
            
            # Draw rectangle for new bounding box
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
 
        # Display tracker type on frame
        cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);
     
        # Display FPS on frame
        cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);
 
        # Display result
        cv2.imshow("Tracking", frame)
 
        # Exit if ESC pressed
        k = cv2.waitKey(1) & 0xff
        if k == 27 : break

(640, 360)
